# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import api_key2

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
city = pd.read_csv("city_data.csv")
city.head()



,Unnamed: 0,City Name,Temp,Feels Like,Max Temp,Min Temp,Pressure,Humidity,Visibility,Latitude,Longitude,Cloudiness,Date,Country,Wind Speed
0,0,Mataura,283.15,282.97,283.15,283.15,1008,91,10000,-46.1927,168.8643,100,1618683941,NZ,1.34
1,1,Thompson,274.15,269.41,274.15,274.15,1017,64,10000,55.7435,-97.8558,90,1618684071,CA,5.14
2,2,Batagay-Alyta,259.32,253.85,259.32,259.32,991,91,9250,67.8006,130.4114,100,1618684052,RU,2.43
3,3,Carballo,288.30,287.20,289.15,287.04,1020,51,10000,43.2130,-8.6910,0,1618684162,ES,2.57
4,4,Bredasdorp,285.15,284.86,285.15,285.15,1017,94,10000,-34.5322,20.0403,27,1618683793,ZA,1.03


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
gmaps.configure(api_key=api_key2)

In [24]:
locations = city[["Latitude", "Longitude"]]
heatmax = humidity.max()
humidity = city["Humidity"].astype(float)

figure = gmaps.figure(center = [0,-60] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=3, max_intensity=heatmax)
figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
#city.dropna()
#add 200 to weather, i.e. - 270 = 70F
narrowed = city.loc[(city["Max Temp"] > 270) & (city["Max Temp"] < 280) & (city["Cloudiness"] == 0) & (city['Wind Speed']<=8), :]
narrowed.head()



,Unnamed: 0,City Name,Temp,Feels Like,Max Temp,Min Temp,Pressure,Humidity,Visibility,Latitude,Longitude,Cloudiness,Date,Country,Wind Speed
138,138,Gribanovskiy,279.82,278.69,279.82,279.82,1017,91,10000,51.4579,41.9764,0,1618684201,RU,1.77
185,185,Manzhouli,275.72,273.38,275.72,275.72,1007,53,10000,49.6000,117.4333,0,1618684216,CN,2.29
330,330,Hohhot,273.15,269.65,273.15,273.15,1020,51,10000,40.8106,111.6522,0,1618684268,CN,3.00
349,349,Shunyi,278.15,276.84,278.15,278.15,1019,60,10000,40.1250,116.6453,0,1618684128,CN,1.71
375,375,Tromsø,278.15,274.94,278.15,278.15,1032,75,10000,69.6496,18.9570,0,1618684284,NO,4.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [97]:
import numpy as np
#Create Dataframe called hotel_df
hotel_df = narrowed.reset_index()
#Create empty hotels list
hotels = []

#Create parameters for search, 5000m etc.
parameters = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": api_key2}

#Use the location we have to select hotels
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    
    #Make sure location changes every iteration
    parameters["location"] = f"{lat},{lon}"
    
    #Add base URL for search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    
    #Create request with... request!
    request = requests.get(base_url, params=parameters)
    #print(request.json())
    request = request.json()
    hotels.append(request["results"][0]["name"])
    #try:
        #print(request["results"][0]["name"])
        #hotels.append(requests["results"][0]["name"])
   # except:
      #  pass
    #Pretty print
    #print(json.dumps(request, indent=4, sort_keys=True))
    
    #Try and skip any that don't work with try and except
array_of_names = np.array(hotels)
hotel_df["Hotel Name"] = array_of_names        
hotel_df.head()

,index,Unnamed: 0,City Name,Temp,Feels Like,Max Temp,Min Temp,Pressure,Humidity,Visibility,Latitude,Longitude,Cloudiness,Date,Country,Wind Speed,Hotel Name
0,138,138,Gribanovskiy,279.82,278.69,279.82,279.82,1017,91,10000,51.4579,41.9764,0,1618684201,RU,1.77,"Gostinitsa ""Meridian"""
1,185,185,Manzhouli,275.72,273.38,275.72,275.72,1007,53,10000,49.6000,117.4333,0,1618684216,CN,2.29,Manzhouli Shangri-La Hotel
2,330,330,Hohhot,273.15,269.65,273.15,273.15,1020,51,10000,40.8106,111.6522,0,1618684268,CN,3.00,"Shangri-La Hotel, Hohhot"
3,349,349,Shunyi,278.15,276.84,278.15,278.15,1019,60,10000,40.1250,116.6453,0,1618684128,CN,1.71,Peking Yard Hostel
4,375,375,Tromsø,278.15,274.94,278.15,278.15,1032,75,10000,69.6496,18.9570,0,1618684284,NO,4.12,Clarion Collection Hotel With


In [95]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [98]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))